In [1]:
import os
from operator import itemgetter
from pydantic import BaseModel, Field
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
import json

In [2]:
embedder = NVIDIAEmbeddings(
    model="nvidia/nv-embedqa-e5-v5"
)

llm = ChatNVIDIA(
    model="openai/gpt-oss-120b", 
    temperature=0.1,
    max_tokens=1024
)

C:\Users\user\AppData\Local\Temp\ipykernel_23968\3149388274.py:5: DeprecationWarning: The 'max_tokens' parameter is deprecated and will be removed in a future version. Please use 'max_completion_tokens' instead.
  llm = ChatNVIDIA(


In [3]:
class EvalScore(BaseModel):
    faithfulness: int = Field(description="Score 1-5: Does the answer stay true to context?")
    relevance: int = Field(description="Score 1-5: Does the answer address the question?")
    reasoning: str = Field(description="Reason for the scores")
    
class GuardrailOutput(BaseModel):
    is_relevant: bool = Field(description="Is the question about the document context?")
    reasoning: str = Field(description="Brief reason")


In [4]:
guardrail_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a security filter. Determine if this question is about contract analysis."),
    ("human", "{question}")
])
guardrail_chain = guardrail_prompt | llm.with_structured_output(GuardrailOutput)

In [5]:
def process_uploaded_file(file_path):
    ext = os.path.splitext(file_path)[1].lower()
    loader = PyPDFLoader(file_path) if ext == '.pdf' else Docx2txtLoader(file_path)
    
    splits = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(loader.load())
    
    # Force a fresh collection to avoid data mixing
    vectorstore = Chroma.from_documents(
        documents=splits, 
        embedding=embedder,
        collection_name="contract_store"
    )
    return vectorstore.as_retriever()

In [6]:
def get_rag_chain(retriever):
    contextual_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a contract assistant. Use the context to answer the question."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "Context: {context}\n\nQuestion: {question}")
    ])

    return (
        RunnableParallel({
            "context": itemgetter("question") | retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)),
            "question": itemgetter("question"),
            "history": itemgetter("history")
        })
        | contextual_prompt 
        | llm 
        | StrOutputParser()
    )

In [7]:
# Use a prompt that explicitly asks for JSON
eval_prompt = ChatPromptTemplate.from_template("""
your task is to evaluate the following RAG response:

Context: 
{context}

Answer: 
{answer}

Respond ONLY with a valid JSON object in this exact format:
{{
  "faithfulness": <int 1-5>,
  "relevance": <int 1-5>,
  "reasoning": "<string>"
}}
""")

# Simple chain — no with_structured_output
eval_chain = eval_prompt | llm | StrOutputParser()

In [8]:
retriever = process_uploaded_file(r"C:\Users\user\Downloads\MarcelenoEskanderCV (5).pdf")

test_input = {
    "question": "What are the main experience highlights?",
    "history": []
}

retrieval_output = RunnableParallel({
    "context": itemgetter("question") | retriever | (lambda docs: "\n\n".join(d.page_content for d in docs)),
    "question": itemgetter("question")
}).invoke(test_input)

answer = get_rag_chain(retriever).invoke(test_input)

if not answer:
    print("❌ Error: The RAG chain failed to generate an answer.")
else:
    try:
        response = eval_chain.invoke({
            "context": retrieval_output["context"],
            "answer": answer
        })
        
        evaluation = EvalScore(**json.loads(response))

        print(f"--- Q&A ---\nQ: {test_input['question']}\nA: {answer}\n")
        print(f"--- EVALUATION ---")
        print(f"Faithfulness: {evaluation.faithfulness}/5")
        print(f"Relevance:     {evaluation.relevance}/5")
        print(f"Reasoning:     {evaluation.reasoning}")

    except Exception as e:
        print(f"❌ Evaluation Error: {e}")

--- Q&A ---
Q: What are the main experience highlights?
A: **Main Experience Highlights**

| Area | Highlights |
|------|------------|
| **Training & Certifications** | • NVIDIA DLI Generative AI (Beginner) – ITI, Dec 2025  <br>• “Building RAG Agents with LLMs” – NVIDIA, Nov 2025 |
| **Key Projects** | **MedAI – Intelligent Medical Assistant**  <br>• Lead AI Architect & Backend Engineer <br>• Designed a state‑machine‑driven agent orchestration layer with **LangGraph** <br>• Built a “Medical Gatekeeper” supervisor for intent classification, safety filtering and routing to RAG, web‑search, or LLM pathways <br>• Implemented a high‑precision RAG pipeline: semantic chunking + **ChromaDB** for medical PDF retrieval <br>• Integrated real‑time web search via **Tavily API** with time‑range filters for up‑to‑date clinical guidelines <br>• Ensured session persistence and fault‑tolerance with **Redis**‑backed storage across distributed nodes <br>• Tech stack: Python, LangChain, LangGraph, OpenAI G